In [1]:
import sqlite3
import pandas as pd
from tqdm.notebook import tqdm

## NPPES data

db = sqlite3.connect('data/nppes_lite.sqlite')

for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20210207.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk = chunk[chunk['provider_business_mailing_address_state_name']=='TN']
    chunk.to_sql('nppes_raw', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

## 2017 Hop Teaming dataset

db = sqlite3.connect('data/nppes_lite.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2017.csv', chunksize = 10000)):
    chunk = chunk[chunk['transaction_count']>=50]
    chunk = chunk[chunk['average_day_wait']<50]
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('hop_team', db, if_exists = 'append', index = False)

## Classification
National Uniform Claim Committee

for chunk in tqdm(pd.read_csv('data/nucc_taxonomy_210.csv',
                              chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('nucc_taxonomy', db, if_exists = 'append', index = False)   # Append the chunk to nucc_taxonomy table

## Zip code to CBSA
using the Business Zip code

db = sqlite3.connect('data/nppes_lite.sqlite')

for chunk in tqdm(pd.read_csv('data/ZIP_CBSA_122020.csv',
                         dtype={'ZIP':str}, #convert ZIP col to str to preserve leading zeros
                         chunksize=1000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('zip_cbsa', db, if_exists = 'append', index = False)   # Append the chunk to zip_cbsa table

db.close()

In [2]:
#could merge within the for loop